In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [2]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True, min_df=10, max_df=0.05)
vectorizer.fit(newsgroups_train.data)

C:\Users\danil\miniconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


CountVectorizer(binary=True, max_df=0.05, min_df=12,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

In [5]:
len(vectorizer.vocabulary_)

8963

In [6]:
vectorizer.vocabulary_

{'wondering': 8843,
 'enlighten': 3078,
 'car': 1612,
 'saw': 7149,
 'door': 2829,
 'sports': 7651,
 'looked': 4949,
 'late': 4746,
 '60s': 446,
 'early': 2925,
 '70s': 482,
 'doors': 2830,
 'small': 7513,
 'addition': 662,
 'bumper': 1513,
 'separate': 7294,
 'rest': 6893,
 'body': 1370,
 'model': 5327,
 'engine': 3065,
 'specs': 7619,
 'production': 6369,
 'history': 4030,
 'info': 4314,
 'fair': 3323,
 'brave': 1441,
 'souls': 7576,
 'upgraded': 8477,
 'si': 7411,
 'clock': 1859,
 'oscillator': 5797,
 'shared': 7352,
 'experiences': 3244,
 'poll': 6173,
 'send': 7277,
 'brief': 1456,
 'message': 5208,
 'procedure': 6352,
 'speed': 7625,
 'cpu': 2308,
 'rated': 6608,
 'add': 659,
 'cards': 1618,
 'adapters': 656,
 'heat': 3973,
 'hour': 4093,
 'usage': 8493,
 'floppy': 3494,
 'disk': 2753,
 'functionality': 3629,
 '800': 500,
 'floppies': 3493,
 'especially': 3128,
 'requested': 6848,
 'days': 2454,
 'network': 5538,
 'knowledge': 4693,
 'base': 1200,
 'upgrade': 8476,
 'haven': 3945

In [7]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 8963)

In [20]:
X_train.nnz / np.prod(X_train.shape)

0.004860444972339391

### ДЗ

In [10]:
import sys
def _print(string):
  sys.stdout.write(string)
  sys.stdout.flush()
import time

In [12]:
#step 1
n_kw, n_dk, n_k = np.zeros((20, 101322)), np.zeros((11314, 20)), np.zeros((20,))
#step 2
xnz = X_train.nonzero()
xnzlen = X_train.nonzero()[0].shape[0]
tags = np.random.choice(20, xnzlen)
for i in range(xnzlen):
  k, d, w = tags[i], xnz[0][i], xnz[1][i]
  n_kw[k, w] += 1
  n_dk[d, k] += 1
  n_k[k] += 1
#step 3
def main_algo(a_k, b, epoch_num = 10):
  for epoch in range(epoch_num):
    start_time = time.time()
    for i in range(xnzlen):
      if i % 10 ** 5 == 0:
        _print("=")
      k, d, w = tags[i], xnz[0][i], xnz[1][i]
      n_kw[k, w] -= 1
      n_dk[d, k] -= 1
      n_k[k] -= 1
      p_k = (n_dk[d] + a_k) * (n_kw[:, w] + b[w]) / (n_k + np.sum(b))
      p_k /= np.sum(p_k)
      tags[i] = np.random.choice(20, p = p_k)
      k = tags[i]
      n_kw[k, w] += 1
      n_dk[d, k] += 1
      n_k[k] += 1
    _print('\nepoch %d took %d seconds\n' % (epoch + 1, time.time() - start_time))
a_k = np.full((20,), 1.) # не понял чем инициализировать
b = np.full((101322,), 1. ) # не понял чем инициализировать
main_algo(a_k, b, 60)

=====
epoch 1 took 54 seconds
=====
epoch 2 took 56 seconds
=====
epoch 3 took 52 seconds
=====
epoch 4 took 53 seconds
=====
epoch 5 took 52 seconds
=====
epoch 6 took 53 seconds
=====
epoch 7 took 53 seconds
=====
epoch 8 took 54 seconds
=====
epoch 9 took 53 seconds
=====
epoch 10 took 54 seconds
=====
epoch 11 took 56 seconds
=====
epoch 12 took 51 seconds
=====
epoch 13 took 51 seconds
=====
epoch 14 took 51 seconds
=====
epoch 15 took 51 seconds
=====
epoch 16 took 53 seconds
=====
epoch 17 took 54 seconds
=====
epoch 18 took 52 seconds
=====
epoch 19 took 52 seconds
=====
epoch 20 took 55 seconds
=====
epoch 21 took 55 seconds
=====
epoch 22 took 52 seconds
=====
epoch 23 took 52 seconds
=====
epoch 24 took 53 seconds
=====
epoch 25 took 52 seconds
=====
epoch 26 took 53 seconds
=====
epoch 27 took 52 seconds
=====
epoch 28 took 52 seconds
=====
epoch 29 took 53 seconds
=====
epoch 30 took 52 seconds
=====
epoch 31 took 52 seconds
=====
epoch 32 took 52 seconds
=====
epoch 33 to

In [18]:
words=np.argsort(n_kw, axis=1)[:, -11:-1]

for i in range(20):
    arr=np.zeros((1, X_train.shape[1]))
    for j in words[i]:
        arr[0,j]=1
    print("{} | {}".format(i + 1, '  '.join(vectorizer.inverse_transform(arr)[0])))

1 | 11  12  14  15  16  17  18  20  23  25
2 | away  clear  example  given  hand  important  means  person  reason  simply
3 | code  files  ftp  graphics  line  program  software  user  version  write
4 | armenian  armenians  children  history  house  president  states  today  turkish  united
5 | 100  condition  cost  interested  low  offer  price  sell  shipping  space
6 | care  getting  happen  health  medical  pay  problems  usually  won  wouldn
7 | 11  1989  1992  21  75  89  92  ca  david  john
8 | games  hockey  league  play  player  players  season  team  teams  win
9 | away  face  fun  head  hey  left  mean  start  thought  went
10 | advance  appreciated  color  display  hi  monitor  running  screen  video  window
11 | ago  bad  guess  guy  oh  pretty  remember  seen  sorry  yes
12 | arab  arabs  israeli  israelis  jewish  jews  land  peace  policy  war
13 | chip  clipper  encryption  government  keys  nsa  phone  secret  secure  security
14 | country  crime  gun  guns  law  la

In [22]:
'|'.join(newsgroups_train.target_names)

'alt.atheism|comp.graphics|comp.os.ms-windows.misc|comp.sys.ibm.pc.hardware|comp.sys.mac.hardware|comp.windows.x|misc.forsale|rec.autos|rec.motorcycles|rec.sport.baseball|rec.sport.hockey|sci.crypt|sci.electronics|sci.med|sci.space|soc.religion.christian|talk.politics.guns|talk.politics.mideast|talk.politics.misc|talk.religion.misc'

Если предобработать исходные данные, то полученные теги легко интерпритируются, например 
3 - comp.graphics, 17 - soc.religion.christian, 8 - rec.sport.hockey итп.